In [1]:
!pip3 install numpy pillow torch torchvision

In [2]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
image, label = train_data[0]

In [6]:
image.size()

torch.Size([3, 32, 32])

In [7]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [8]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)  # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2)  # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5)  # (24, 10, 10) -> (24, 5, 5) -> Flatten (24 * 5 * 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(30):
  print(f'Training epoch {epoch}...')

  running_loss = 0.0

  for i, data in enumerate(train_loader):
    inputs, labels = data

    optimizer.zero_grad()

    outputs = net(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.1784
Training epoch 1...
Loss: 1.7338
Training epoch 2...
Loss: 1.5065
Training epoch 3...
Loss: 1.3877
Training epoch 4...
Loss: 1.2990
Training epoch 5...
Loss: 1.2174
Training epoch 6...
Loss: 1.1458
Training epoch 7...
Loss: 1.0843
Training epoch 8...
Loss: 1.0264
Training epoch 9...
Loss: 0.9835
Training epoch 10...
Loss: 0.9393
Training epoch 11...
Loss: 0.8999
Training epoch 12...
Loss: 0.8667
Training epoch 13...
Loss: 0.8368
Training epoch 14...
Loss: 0.8041
Training epoch 15...
Loss: 0.7768
Training epoch 16...
Loss: 0.7454
Training epoch 17...
Loss: 0.7248
Training epoch 18...
Loss: 0.6953
Training epoch 19...
Loss: 0.6716
Training epoch 20...
Loss: 0.6474
Training epoch 21...
Loss: 0.6238
Training epoch 22...
Loss: 0.6009
Training epoch 23...
Loss: 0.5795
Training epoch 24...
Loss: 0.5548
Training epoch 25...
Loss: 0.5370
Training epoch 26...
Loss: 0.5133
Training epoch 27...
Loss: 0.4919
Training epoch 28...
Loss: 0.4726
Training epoch 29...
Los

In [11]:
torch.save(net.state_dict(), 'trained_net.pth')

In [12]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\ahaan\AppData\Local\Temp\ipykernel_34848\1690949604.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [13]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 69.19%


In [19]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0)
  return image

image_paths = ['example1.jpg', 'example2.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _, predicted = torch.max(output.data, 1)
    print(f'Predicted class: {class_names[predicted]}')

Predicted class: dog
Predicted class: ship
